
## Overview

Spark can easily read partitioned data - in this example I've added some paritioned data to my sample git repository. In this case its partitioned by YEAR. When spark reads in the data from /sample2/partitioned/ the resulting data frame will have a YEAR column in addition to those columns in the CSV.

```
sample2
├── partitioned
│   ├── year=2022
│   │   └── data.csv
│   ├── year=2023
│   │   └── data.csv
│   └── year=2024
│       └── data.csv
```

### Clone the data repository so we have a local copy

In [0]:
%sh

if [ ! -d "/tmp/data" ]; then
  echo "Cloning data"
  git clone https://github.com/prule/data.git /tmp/data
else
  echo "Pulling data"
  cd /tmp/data && git pull
fi


Pulling data
Already up to date.


### Check data location

In [0]:
!ls -al /tmp/data/sample2/partitioned/

total 20
drwxr-xr-x 5 root root 4096 May 11 10:44  .
drwxr-xr-x 4 root root 4096 May 11 10:44  ..
drwxr-xr-x 2 root root 4096 May 11 10:44 'year=2022'
drwxr-xr-x 2 root root 4096 May 11 10:44 'year=2023'
drwxr-xr-x 2 root root 4096 May 11 10:44 'year=2024'


### Read partitioned data into a spark dataframe

The schema for the dataframe is inferred so automatic typing occurs.

In [0]:
# File location and type
file_location = "file:/tmp/data/sample2/partitioned/"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

amount,type,year
4,A,2023
5,B,2023
6,C,2023
1,A,2022
2,B,2022
3,C,2022
7,A,2024
8,B,2024
9,C,2024


In [0]:
df.printSchema()

root
 |-- amount: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)



### Create a view on the dataframe

In [0]:
# Create a view or table

temp_table_name = "data_csv"

df.createOrReplaceTempView(temp_table_name)

### Query the view with SQL

In [0]:
%sql

select * from `data_csv`

amount,type,year
4,A,2023
5,B,2023
6,C,2023
1,A,2022
2,B,2022
3,C,2022
7,A,2024
8,B,2024
9,C,2024


### Write the dataframe to disk

This time using a different column to partition it. Here I'm saving it to temporary storage but it is possible to save it to the DBFS area and have it persist across cluster restarts as well as allow various users across different notebooks to query this data.

In [0]:
df.write.partitionBy("type").parquet(path='file:/tmp/output/sample-data.parquet', mode='overwrite')

In [0]:
%sh
ls -al /tmp/output/sample-data.parquet

total 24
drwxr-xr-x 5 root root 4096 May 11 10:59 .
drwxr-xr-x 3 root root 4096 May 11 10:59 ..
-rw-r--r-- 1 root root    8 May 11 10:59 ._SUCCESS.crc
-rw-r--r-- 1 root root    0 May 11 10:59 _SUCCESS
drwxr-xr-x 2 root root 4096 May 11 10:59 type=A
drwxr-xr-x 2 root root 4096 May 11 10:59 type=B
drwxr-xr-x 2 root root 4096 May 11 10:59 type=C


### Invalid partitioning

If the partitioned data structure isn't correct, there'll be problems. In this folder we expect partitions by "year" - meaning all the folders should be in the format "year=nnnn" - but here we have a non-conforming directory "invalid". Spark won't like this and will error.

In [0]:
%sh
ls -al /tmp/data/sample2/partitioned-invalid/

total 24
drwxr-xr-x 6 root root 4096 May 11 10:44 .
drwxr-xr-x 4 root root 4096 May 11 10:44 ..
drwxr-xr-x 2 root root 4096 May 11 10:44 invalid
drwxr-xr-x 2 root root 4096 May 11 10:44 year=2022
drwxr-xr-x 2 root root 4096 May 11 10:44 year=2023
drwxr-xr-x 2 root root 4096 May 11 10:44 year=2024


In [0]:
# File location and type
file_location = "file:/tmp/data/sample2/partitioned-invalid/"

# Trying to load the invalid data structure gives an error: "Conflicting directory structures detected."
df = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3819069725089261>:5
      2 file_location = "file:/tmp/data/sample2/partitioned-invalid/"
      4 # Trying to load the invalid data structure gives an error: "Conflicting directory structures detected."
----> 5 df = spark.read.format("csv") \
      6   .option("inferSchema", "true") \
      7   .option("header", "true") \
      8   .option("sep", ",") \
      9   .load(file_location)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:302, in DataFrame